# Analysis

**Initializing**

In [412]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

from Brands import Brands
from Analyzer import Analyzer
from Solver import Solver


#an = Analyzer(marketing_mix_segment_weights=move_mkt_weights, last_period=2)
an = Analyzer(last_period=8, sector="Vodites")

brd = Brands(sector="Vodites")

solver = Solver(sector="Vodites")


Vodites
./Exports/TeamExport_A46051_Alpha_M_Period 9.xlsx loaded
No weights provided
Vodites
./Exports/TeamExport_A46051_Alpha_M_Period 9.xlsx loaded
Attributes file:./Attributes/Vodites/attributes_Vodites_9.json


**Percentage expenditure by segment for each product**

In [413]:
df_marketing_mixes = brd.get_marketing_mixes(capped=False)
df_marketing_mixes


,Innovators,Early Adopters,Followers
0,0.149425,0.149425,0.701149
1,0.151394,0.697211,0.151394
2,0.178723,0.202128,0.619149


## Semantic scales

In [414]:
features_name = ["Resolution", "Energy Efficiency", "Carbon Footprint", "Connectivity", "No. of Apps", "Price"]
df_seg_sem = brd.df_segments_semantic[:3][features_name]
df_seg_sem


,Resolution,Energy Efficiency,Carbon Footprint,Connectivity,No. of Apps,Price
Index,,,,,,
Adopters_9,5.13,4.54,4.65,4.89,4.51,4.82
Followers_9,4.66,4.92,2.52,4.27,3.95,4.39
Innovators_9,5.48,3.96,3.93,5.38,3.99,4.86


In [415]:
brd.df_brands_semantic

,Resolution,Energy Efficiency,Carbon Footprint,Connectivity,No. of Apps,Price
MARKET : Vodites,,,,,,
MEMORE,4.86,5.05,4.44,4.52,4.05,4.47
MEOW,4.29,2.94,4.56,4.40,4.14,5.16
REEGO,4.89,5.22,3.24,4.25,4.17,4.47


### Compute Distances between Segments and Products 

#### Compute closest brands for each segment

In [416]:
brd.df_brands_semantic

,Resolution,Energy Efficiency,Carbon Footprint,Connectivity,No. of Apps,Price
MARKET : Vodites,,,,,,
MEMORE,4.86,5.05,4.44,4.52,4.05,4.47
MEOW,4.29,2.94,4.56,4.40,4.14,5.16
REEGO,4.89,5.22,3.24,4.25,4.17,4.47


In [417]:
_ = an.get_n_closest(df_base=df_seg_sem, df_performers=brd.df_brands_semantic, num_top=3, max_distance_1D=6, distance_metric=3)

---------- Adopters_9 ----------
Segment:	 MEMORE
Distance:	 0.9376531148431376
Segment:	 REEGO
Distance:	 0.8999245641783432
Segment:	 MEOW
Distance:	 0.8721188201497215

---------- Followers_9 ----------
Segment:	 REEGO
Distance:	 0.9586212972504712
Segment:	 MEMORE
Distance:	 0.9134374035898786
Segment:	 MEOW
Distance:	 0.8338696779162705

---------- Innovators_9 ----------
Segment:	 MEMORE
Distance:	 0.882536814545801
Segment:	 REEGO
Distance:	 0.8591014267921377
Segment:	 MEOW
Distance:	 0.8590422112621847



#### Compute closest Segments for each Brand

In [418]:
_ = an.get_n_closest(df_base=brd.df_brands_semantic, df_performers=df_seg_sem, max_distance_1D=6)

---------- MEMORE ----------
Segment:	 Adopters_9
Distance:	 0.9376531148431376
Segment:	 Followers_9
Distance:	 0.9134374035898786
Segment:	 Innovators_9
Distance:	 0.882536814545801

---------- MEOW ----------
Segment:	 Adopters_9
Distance:	 0.8721188201497215
Segment:	 Innovators_9
Distance:	 0.8590422112621847
Segment:	 Followers_9
Distance:	 0.8338696779162705

---------- REEGO ----------
Segment:	 Followers_9
Distance:	 0.9586212972504712
Segment:	 Adopters_9
Distance:	 0.8999245641783432
Segment:	 Innovators_9
Distance:	 0.8591014267921377



#### Distances between all points

In [419]:
df_all_sem = brd.get_comprehensive_df_semantic()
index = df_all_sem.index 

distances = an.compute_distance_centroids(df_all_sem, df_all_sem, max_distance_1D=6)[3]


df_out_sem = pd.DataFrame(columns=index, index=index)

for i, start in enumerate(index):
    index_to_search = index[i:]
    for stop in index_to_search:
        df_out_sem.loc[start, stop] = distances[start][stop]
        df_out_sem.loc[stop, start] = distances[start][stop]

df_out_sem

,MEMORE,MEOW,REEGO,Adopters_9,Followers_9,Innovators_9
MEMORE,1.0,0.851101,0.942277,0.937653,0.913437,0.882537
MEOW,0.851101,1.0,0.831498,0.872119,0.83387,0.859042
REEGO,0.942277,0.831498,1.0,0.899925,0.958621,0.859101
Adopters_9,0.937653,0.872119,0.899925,1.0,0.877827,0.926419
Followers_9,0.913437,0.83387,0.958621,0.877827,1.0,0.852881
Innovators_9,0.882537,0.859042,0.859101,0.926419,0.852881,1.0


## Multi Dimensional Scaling

In [420]:
brd.df_segments_mds

,Segment,Period,Autonomy,Sophistication,Economy
Index,,,,,
Adopters_9,Adopters,9,5.22,4.7,-5.44
Followers_9,Followers,9,3.1,4.02,-2.6
Innovators_9,Innovators,9,6.36,5.96,-5.76
Early Adopters_8,Adopters,8,4.68,4.46,-6.84
Followers_8,Followers,8,2.12,3.14,-2.84
Innovators_8,Innovators,8,5.88,5.68,-7.8
Early Adopters_7,Adopters,7,3.52,3.94,-7.76
Followers_7,Followers,7,0.96,2.32,-3.2
Innovators_7,Innovators,7,5.16,5.24,-9.98


In [421]:
brd.df_brands_mds

,Autonomy,Sophistication,Economy
MARKET : Vodites,,,
MEMORE,4.52,3.08,-3.16
MEOW,-0.28,0.82,-7.76
REEGO,3.58,4.58,-3.12


### Compute Distances between Segments and Products 

#### Compute closest brands for each segment

In [422]:
brd.df_segments_mds

,Segment,Period,Autonomy,Sophistication,Economy
Index,,,,,
Adopters_9,Adopters,9,5.22,4.7,-5.44
Followers_9,Followers,9,3.1,4.02,-2.6
Innovators_9,Innovators,9,6.36,5.96,-5.76
Early Adopters_8,Adopters,8,4.68,4.46,-6.84
Followers_8,Followers,8,2.12,3.14,-2.84
Innovators_8,Innovators,8,5.88,5.68,-7.8
Early Adopters_7,Adopters,7,3.52,3.94,-7.76
Followers_7,Followers,7,0.96,2.32,-3.2
Innovators_7,Innovators,7,5.16,5.24,-9.98


In [423]:
df_seg_mds = brd.df_segments_mds[:3][['Autonomy', 'Sophistication', 'Economy']]

_ = an.get_n_closest(df_base=df_seg_mds, df_performers=brd.df_brands_mds, num_top=3, max_distance_1D=40, weighted="eq")

---------- Adopters_9 ----------
Segment:	 REEGO
Distance:	 0.9589553089100835
Segment:	 MEMORE
Distance:	 0.9583846983270176
Segment:	 MEOW
Distance:	 0.8972393557824787

---------- Followers_9 ----------
Segment:	 REEGO
Distance:	 0.9869743842116134
Segment:	 MEMORE
Distance:	 0.9741253019341288
Segment:	 MEOW
Distance:	 0.89969837156523

---------- Innovators_9 ----------
Segment:	 REEGO
Distance:	 0.9411881531208777
Segment:	 MEMORE
Distance:	 0.9380188200607098
Segment:	 MEOW
Distance:	 0.8754096713223695



#### Compute closest Segments for each Brand

In [424]:
_ = an.get_n_closest(df_base=brd.df_brands_mds, df_performers=df_seg_mds, num_top=3, max_distance_1D=40, weighted="eq")

---------- MEMORE ----------
Segment:	 Followers_9
Distance:	 0.9741253019341288
Segment:	 Adopters_9
Distance:	 0.9583846983270176
Segment:	 Innovators_9
Distance:	 0.9380188200607098

---------- MEOW ----------
Segment:	 Followers_9
Distance:	 0.89969837156523
Segment:	 Adopters_9
Distance:	 0.8972393557824787
Segment:	 Innovators_9
Distance:	 0.8754096713223695

---------- REEGO ----------
Segment:	 Followers_9
Distance:	 0.9869743842116134
Segment:	 Adopters_9
Distance:	 0.9589553089100835
Segment:	 Innovators_9
Distance:	 0.9411881531208777



#### Distances between all points

In [425]:
df_all_mds = brd.get_comprehensive_df_mds()
index = df_all_mds.index 

distances = an.compute_distance_centroids(df_all_mds, df_all_mds, weighted="eq", max_distance_1D=40)[3]


df_out_mds = pd.DataFrame(columns=index, index=index)

for i, start in enumerate(index):
    index_to_search = index[i:]
    for stop in index_to_search:
        df_out_mds.loc[start, stop] = distances[start][stop]
        df_out_mds.loc[stop, start] = distances[start][stop]

df_out_mds

,MEMORE,MEOW,REEGO,Adopters_9,Followers_9,Innovators_9
MEMORE,1.0,0.898647,0.974443,0.958385,0.974125,0.938019
MEOW,0.898647,1.0,0.897361,0.897239,0.899698,0.87541
REEGO,0.974443,0.897361,1.0,0.958955,0.986974,0.941188
Adopters_9,0.958385,0.897239,0.958955,1.0,0.947914,0.975043
Followers_9,0.974125,0.899698,0.986974,0.947914,1.0,0.928736
Innovators_9,0.938019,0.87541,0.941188,0.975043,0.928736,1.0


# Forecast analysis

In [426]:
df_seg_sem_fc = an.forecast_df(brd.df_segments_semantic, steps=2)

df_seg_sem_fc = df_seg_sem_fc[(df_seg_sem_fc["Period"]>an.last_period)].drop(labels=["Period", "Segment"], axis=1)
df_seg_sem_fc

,Resolution,Energy Efficiency,Carbon Footprint,Connectivity,No. of Apps,Price
Innovators_11,5.52,4.24,3.77,5.46,4.15,4.24
Adopters_11,5.21,4.9,4.59,4.97,4.59,4.4
Followers_11,4.96,5.08,2.3,4.63,4.17,4.31
Innovators_10,5.5,4.1,3.85,5.42,4.07,4.55
Followers_10,4.81,5.0,2.41,4.45,4.06,4.35
Adopters_10,5.17,4.72,4.62,4.93,4.55,4.61
Innovators_9,5.48,3.96,3.93,5.38,3.99,4.86
Adopters_9,5.13,4.54,4.65,4.89,4.51,4.82
Followers_9,4.66,4.92,2.52,4.27,3.95,4.39


In [427]:
pippo = np.dot([0.75,0.25], [[4.81, 5.0, 2.41, 4.45, 4.06, 4.35], [5.17,4.72, 4.62, 4.93, 4.55, 4.61]])
pippo

array([4.9   , 4.93  , 2.9625, 4.57  , 4.1825, 4.415 ])

In [428]:
df_all_sem = pd.concat([df_seg_sem_fc, brd.df_brands_semantic], join="inner")
index = df_all_sem.index

distances = an.compute_distance_centroids(df_all_sem, df_all_sem, weighted="default", max_distance_1D=6)[3]


df_out_sem_fc = pd.DataFrame(columns=index, index=index)

for i, start in enumerate(index):
    index_to_search = index[i:]
    for stop in index_to_search:
        df_out_sem_fc.loc[start, stop] = distances[start][stop]
        df_out_sem_fc.loc[stop, start] = distances[start][stop]

df_out_sem_fc

,Innovators_11,Adopters_11,Followers_11,Innovators_10,Followers_10,Adopters_10,Innovators_9,Adopters_9,Followers_9,MEMORE,MEOW,REEGO
Innovators_11,1.0,0.923074,0.883307,0.972148,0.87286,0.920349,0.944296,0.912383,0.859938,0.886828,0.827769,0.865716
Adopters_11,0.923074,1.0,0.893556,0.920643,0.889218,0.978719,0.909286,0.957437,0.88117,0.944444,0.838865,0.906459
Followers_11,0.883307,0.893556,1.0,0.878326,0.978723,0.889799,0.867548,0.882254,0.957445,0.907008,0.814373,0.945134
Innovators_10,0.972148,0.920643,0.878326,1.0,0.870088,0.925296,0.972148,0.924041,0.859092,0.888076,0.845111,0.865217
Followers_10,0.87286,0.889218,0.978723,0.870088,1.0,0.887507,0.861649,0.881922,0.978723,0.912721,0.825141,0.956314
Adopters_10,0.920349,0.978719,0.889799,0.925296,0.887507,1.0,0.920196,0.978719,0.881381,0.944919,0.856104,0.905503
Innovators_9,0.944296,0.909286,0.867548,0.972148,0.861649,0.920196,1.0,0.926419,0.852881,0.882537,0.859042,0.859101
Adopters_9,0.912383,0.957437,0.882254,0.924041,0.881922,0.978719,0.926419,1.0,0.877827,0.937653,0.872119,0.899925
Followers_9,0.859938,0.88117,0.957445,0.859092,0.978723,0.881381,0.852881,0.877827,1.0,0.913437,0.83387,0.958621
MEMORE,0.886828,0.944444,0.907008,0.888076,0.912721,0.944919,0.882537,0.937653,0.913437,1.0,0.851101,0.942277


In [429]:
df_seg_mds_fc = an.forecast_df(brd.df_segments_mds, steps=2)

df_seg_mds_fc = df_seg_mds_fc[(df_seg_mds_fc["Period"]>an.last_period)][['Autonomy', 'Sophistication', 'Economy']]
df_seg_mds_fc

,Autonomy,Sophistication,Economy
Innovators_11,7.32,6.52,-1.68
Adopters_11,6.3,5.18,-2.64
Followers_11,5.06,5.78,-2.12
Innovators_10,6.84,6.24,-3.72
Followers_10,4.08,4.9,-2.36
Adopters_10,5.76,4.94,-4.04
Innovators_9,6.36,5.96,-5.76
Adopters_9,5.22,4.7,-5.44
Followers_9,3.1,4.02,-2.6


In [430]:
brd.df_brands_mds

,Autonomy,Sophistication,Economy
MARKET : Vodites,,,
MEMORE,4.52,3.08,-3.16
MEOW,-0.28,0.82,-7.76
REEGO,3.58,4.58,-3.12


In [431]:
df_all_mds = pd.concat([df_seg_mds_fc, brd.df_brands_mds], join="inner")
index = df_all_mds.index

distances = an.compute_distance_centroids(df_all_mds, df_all_mds, weighted="eq", max_distance_1D=40)[3]


df_out_mds_fc = pd.DataFrame(columns=index, index=index)

for i, start in enumerate(index):
    index_to_search = index[i:]
    for stop in index_to_search:
        df_out_mds_fc.loc[start, stop] = distances[start][stop]
        df_out_mds_fc.loc[stop, start] = distances[start][stop]

df_out_mds_fc

,Innovators_11,Adopters_11,Followers_11,Innovators_10,Followers_10,Adopters_10,Innovators_9,Adopters_9,Followers_9,MEMORE,MEOW,REEGO
Innovators_11,1.0,0.972021,0.965093,0.969482,0.946801,0.95323,0.938964,0.932515,0.927969,0.932509,0.837201,0.935734
Adopters_11,0.972021,1.0,0.978748,0.976809,0.967451,0.978066,0.953573,0.956133,0.950868,0.959563,0.8642,0.959204
Followers_11,0.965093,0.978748,1.0,0.964822,0.980676,0.968108,0.94415,0.949555,0.961352,0.957517,0.866985,0.968941
Innovators_10,0.969482,0.976809,0.964822,1.0,0.95156,0.975172,0.969482,0.959292,0.935176,0.942842,0.85829,0.946491
Followers_10,0.946801,0.967451,0.980676,0.95156,1.0,0.965702,0.938964,0.952509,0.980676,0.97061,0.883796,0.986081
Adopters_10,0.95323,0.978066,0.968108,0.975172,0.965702,1.0,0.969866,0.978066,0.954366,0.965324,0.881595,0.965454
Innovators_9,0.938964,0.953573,0.94415,0.969482,0.938964,0.969866,1.0,0.975043,0.928736,0.938019,0.87541,0.941188
Adopters_9,0.932515,0.956133,0.949555,0.959292,0.952509,0.978066,0.975043,1.0,0.947914,0.958385,0.897239,0.958955
Followers_9,0.927969,0.950868,0.961352,0.935176,0.980676,0.954366,0.928736,0.947914,1.0,0.974125,0.899698,0.986974
MEMORE,0.932509,0.959563,0.957517,0.942842,0.97061,0.965324,0.938019,0.958385,0.974125,1.0,0.898647,0.974443


## MORE - Next Period

In [432]:
df_seg_sem_fc.loc[f"Followers_{an.last_period+1}"]

Resolution           4.66
Energy Efficiency    4.92
Carbon Footprint     2.52
Connectivity         4.27
No. of Apps          3.95
Price                4.39
Name: Followers_9, dtype: object

In [433]:
more_features = np.array([70, 65, 35, 7, 53, 0])
#more_features = brd.get_features("MORE").drop(labels="Base Cost", axis=1).values.flatten()

In [434]:
solver.regress_semantic(more_features)

array([4.87324729, 4.96145217, 4.75193344, 1.        , 4.05575611,
       0.44151357])

In [435]:
df_seg_sem_fc.loc[f"Followers_{an.last_period+1}"]

Resolution           4.66
Energy Efficiency    4.92
Carbon Footprint     2.52
Connectivity         4.27
No. of Apps          3.95
Price                4.39
Name: Followers_9, dtype: object

In [436]:
from Utils import combined_error

followers_semantic_mk = df_seg_sem_fc.loc[f"Followers_{an.last_period+1}"]
followers_semantic_mk = pippo
explorers_mds_mk = [0,0,0]

prices_array = np.linspace(536*0.75, 536*1.3, num=8)
features = more_features.copy()

for price in prices_array:
    price = round(price)
    print(f"Price {price}")
    no_price_features = np.delete(features, -1)
    new_features = np.append(no_price_features, price)
    res = combined_error(features=new_features, ideal_semantic=followers_semantic_mk, ideal_mds=[0,0,0], semantic_weights=brd.rel_importance_features, mds_weights=[1/3, 1/3,1/3], error_weights=[1,0], model=solver)
    print(f"Combined_error: {res}")
    contr = an.compute_contribution(price, 289, [17/38,12/38,9/38]) ### CHANGE THIS
    print(f"Contribution: {contr}")
    print()



Price 402
semantic_error 0.33925646545656263 mds_error 0.0
Combined_error: 0.6785129309131253
Contribution: -37

Price 444
semantic_error 0.3332602642235831 mds_error 0.0
Combined_error: 0.6665205284471662
Contribution: -11

Price 486
semantic_error 0.32897769291976076 mds_error 0.0
Combined_error: 0.6579553858395215
Contribution: 16

Price 528
semantic_error 0.32647619444515086 mds_error 0.0
Combined_error: 0.6529523888903017
Contribution: 42

Price 570
semantic_error 0.325796797117637 mds_error 0.0
Combined_error: 0.651593594235274
Contribution: 68

Price 613
semantic_error 0.3270005865601062 mds_error 0.0
Combined_error: 0.6540011731202124
Contribution: 95

Price 655
semantic_error 0.33001158627329885 mds_error 0.0
Combined_error: 0.6600231725465977
Contribution: 122

Price 697
semantic_error 0.33478705905504746 mds_error 0.0
Combined_error: 0.6695741181100949
Contribution: 148



In [437]:
no_price_features = np.delete(features, -1)
new_features = np.append(no_price_features, 580)

In [448]:
brd.get_semantic("MORE")

,Resolution,Energy Efficiency,Carbon Footprint,Connectivity,No. of Apps,Price
MARKET : Vodites,,,,,,


In [447]:
pippo

array([4.9   , 4.93  , 2.9625, 4.57  , 4.1825, 4.415 ])

In [438]:
new_features_more = new_features # Change price

semantic_regressed_more = np.array(solver.regress_semantic(new_features_more))
semantic_regressed_more = [round(sem, 1) for sem in semantic_regressed_more]
semantic_regressed_more

[4.9, 5.0, 4.8, 1.0, 4.1, 4.5]

In [439]:
an.compute_distance_centroids(semantic_regressed_more, followers_semantic_mk)[3]

{'observation': {'centroid': 0.6736768998087248}}

In [440]:
an.compute_distance_centroids(semantic_regressed_more, followers_semantic_mk)[4]


{'observation': {'centroid': array([8.8817842e-16, 7.0000000e-02, 1.8375000e+00, 3.5700000e+00,
         8.2500000e-02, 8.5000000e-02])}}

In [441]:
mds_more = solver.regress_mds(new_features_more)
mds_more = [round(mds) for mds in mds_more] 
mds_more

[0, 0, 0]

In [442]:
an.compute_distance_centroids(mds_more, explorers_mds_mk, weighted="eq", max_distance_1D=40)[3]

d:\Uni\Anno III\UCSD\112\Markstrat\Code\Utils.py:271: RuntimeWarning: invalid value encountered in divide
  relative_distance = manhattan_distance / feat_values_array
d:\Uni\Anno III\UCSD\112\Markstrat\Code\Utils.py:283: RuntimeWarning: invalid value encountered in divide
  relative_metric = (current_centroid - feat_values_array)/feat_values_array


{'observation': {'centroid': 1.0}}

In [443]:
brd.move_mds

,Autonomy,Sophistication,Economy
MARKET : Vodites,,,


In [444]:
explorers_mds_mk

[0, 0, 0]

In [445]:
an.compute_distance_centroids(mds_more, explorers_mds_mk, weighted="eq", max_distance_1D=40)[4]

d:\Uni\Anno III\UCSD\112\Markstrat\Code\Utils.py:271: RuntimeWarning: invalid value encountered in divide
  relative_distance = manhattan_distance / feat_values_array
d:\Uni\Anno III\UCSD\112\Markstrat\Code\Utils.py:283: RuntimeWarning: invalid value encountered in divide
  relative_metric = (current_centroid - feat_values_array)/feat_values_array


{'observation': {'centroid': array([0, 0, 0], dtype=int64)}}

In [455]:
from Scraper import Scraper


scr = Scraper()
scr.download_attributes(sector="Vodites", period=10)

---------- Logged In ----------
Create
---------- Accessed Regression Page ----------
{'Brands': {'1056035': {'BrdId': 1056035, 'BrdName': 'REEGO', 'FirId': 2, 'RetailPrice': 560, 'AttLevels': {'6': 70, '7': 70, '8': 22, '9': 7, '10': 55}, 'SemMDSPerceptions': {'6': 4.218999862670898, '7': 4.867000102996826, '8': 5.3379998207092285, '9': 3.180999994277954, '10': 4.243000030517578, '11': 4.201000213623047}}, '1062831': {'BrdId': 1062831, 'BrdName': 'MEMORE', 'FirId': 1, 'RetailPrice': 570, 'AttLevels': {'6': 70, '7': 65, '8': 35, '9': 7, '10': 53}, 'SemMDSPerceptions': {'6': 4.473999977111816, '7': 4.861000061035156, '8': 5.046999931335449, '9': 4.39300012588501, '10': 4.521999835968018, '11': 4.051000118255615}}}, 'DimSlopIntercept': {'6': {'Item1': 39.2156867980957, 'Item2': 394.54901123046875}, '7': {'Item1': 0, 'Item2': 70}, '8': {'Item1': 17.182130813598633, 'Item2': -21.718212127685547}, '9': {'Item1': 10.726072311401367, 'Item2': -12.119636535644531}, '10': {'Item1': 0, 'Item2': 